
<center><h2>Lookalike Model</h2></center>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## Display all the columns of the dataframe
pd.pandas.set_option('display.max_columns',None)

- **Customers**, **Products**, and **Transactions** contain complementary information.  
- So, we gather all the required information (Customers,Products, and Transactions data) into a single dataset for analysis and similarity calculation..  



In [2]:
# Loading the datasets
customers_data = pd.read_csv("Customers.csv")
products_data = pd.read_csv("Products.csv")
transactions_data = pd.read_csv("Transactions.csv")

# Merging Transactions with Customers
transactions_customers = pd.merge(transactions_data, customers_data, on="CustomerID", how="left")

# Merging the transactions_customers with Products
dataset = pd.merge(transactions_customers, products_data, on="ProductID", how="left")

In [3]:
dataset.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,CustomerName,Region,SignupDate,ProductName,Category,Price_y
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,Andrea Jenkins,Europe,2022-12-03,ComfortLiving Bluetooth Speaker,Electronics,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,Brittany Harvey,Asia,2024-09-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,Kathryn Stevens,Europe,2024-04-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,Travis Campbell,South America,2024-04-11,ComfortLiving Bluetooth Speaker,Electronics,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,Timothy Perez,Europe,2022-03-15,ComfortLiving Bluetooth Speaker,Electronics,300.68


Why there are two Price columns?  
This happend during the merge operation from both the datasets(Transactions and Products)  
- **_x**: it had occured from the left dataset (Transactions).
- **_y**: and it was from the right dataset (Products).

In [4]:
# verifying wheteher Price_x and Price_y had any differences.
price_mismatch = dataset[dataset["Price_x"] != dataset["Price_y"]]
print("Price mismatches:\n", price_mismatch)

Price mismatches:
 Empty DataFrame
Columns: [TransactionID, CustomerID, ProductID, TransactionDate, Quantity, TotalValue, Price_x, CustomerName, Region, SignupDate, ProductName, Category, Price_y]
Index: []


- According to the output there is no difference between **Price_x** (from the Transactions dataset) and **Price_y** (from the Products dataset). This indicates that there is consistency in **Price** feature across both the datasets.  
- So, there were no mismatches; we can drop one of the redundant columns.  

In [5]:
# Dropping the redundant column (Price_y)
dataset.drop(columns=["Price_y"], inplace=True)

# Renaming the remained column (Price_x) to "Price"
dataset.rename(columns={"Price_x": "Price"}, inplace=True)

# Displaying the cleaned dataset
print(dataset.head())


  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3        T00272      C0087      P067  2024-03-26 22:55:37         2   
4        T00363      C0070      P067  2024-03-21 15:10:10         3   

   TotalValue   Price     CustomerName         Region  SignupDate  \
0      300.68  300.68   Andrea Jenkins         Europe  2022-12-03   
1      300.68  300.68  Brittany Harvey           Asia  2024-09-04   
2      300.68  300.68  Kathryn Stevens         Europe  2024-04-04   
3      601.36  300.68  Travis Campbell  South America  2024-04-11   
4      902.04  300.68    Timothy Perez         Europe  2022-03-15   

                       ProductName     Category  
0  ComfortLiving Bluetooth Speaker  Electronics  
1  ComfortLiving Bluetooth Speaker  Electronics  
2  Comfo

In [6]:
dataset.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price,CustomerName,Region,SignupDate,ProductName,Category
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,Andrea Jenkins,Europe,2022-12-03,ComfortLiving Bluetooth Speaker,Electronics
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,Brittany Harvey,Asia,2024-09-04,ComfortLiving Bluetooth Speaker,Electronics
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,Kathryn Stevens,Europe,2024-04-04,ComfortLiving Bluetooth Speaker,Electronics
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,Travis Campbell,South America,2024-04-11,ComfortLiving Bluetooth Speaker,Electronics
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,Timothy Perez,Europe,2022-03-15,ComfortLiving Bluetooth Speaker,Electronics


What is **TotalValue**?  
In the dataset, **TotalValue** represents the total monetary value of a specific transaction.  
It is calculated as:  

<center><h5>TotalValue = Price × Quantity</h5></center>  

Where:  
- **Price**: The price of a single unit of the product involved in the transaction.  
- **Quantity**: The number of units of the product purchased in that transaction.  


### Verifying and Correcting 'TotalValue' Field

In [7]:
# 2. Validating 'TotalValue'
incorrect_totalvalue = dataset[dataset["TotalValue"] != dataset["Price"] * dataset["Quantity"]]
print(f"Number of incorrect 'TotalValue' rows: {len(incorrect_totalvalue)}")
if len(incorrect_totalvalue) == 0:
    print("All 'TotalValue' calculations are correct.")

Number of incorrect 'TotalValue' rows: 88


In [8]:
# Correcting 'TotalValue' based on 'Price' and 'Quantity' for rows with discrepancies
dataset.loc[dataset["TotalValue"] != dataset["Price"] * dataset["Quantity"], "TotalValue"] = (
    dataset["Price"] * dataset["Quantity"]
)

# Verifying corrections
incorrect_totalvalue_after_correction = dataset[dataset["TotalValue"] != dataset["Price"] * dataset["Quantity"]]

# Displaying the number of discrepancies after correction
print(f"Number of incorrect 'TotalValue' rows after correction: {len(incorrect_totalvalue_after_correction)}")

# Confirming that the incorrect values has been corrected
if len(incorrect_totalvalue_after_correction) == 0:
    print("All discrepancies in 'TotalValue' have been successfully corrected.")


Number of incorrect 'TotalValue' rows after correction: 0
All discrepancies in 'TotalValue' have been successfully corrected.


### Analyzing the Unique Product, Category, and Region Data

In [9]:
# Displaying all unique product names
unique_products = dataset["ProductName"].unique()

# Converting to a DataFrame for a better readability
unique_products_df = pd.DataFrame(unique_products, columns=["ProductName"])

# Displaying the total number of unique products
print(f"Total Unique Products: {len(unique_products)}")

# Displaying the product names
print("\nList of Unique Product Names:")
print(unique_products_df)


Total Unique Products: 66

List of Unique Product Names:
                        ProductName
0   ComfortLiving Bluetooth Speaker
1                 HomeSense T-Shirt
2             ActiveWear Smartphone
3                  TechPro Textbook
4             TechPro Running Shoes
..                              ...
61           HomeSense Cookware Set
62      BookWorld Bluetooth Speaker
63                 SoundWave Laptop
64      SoundWave Bluetooth Speaker
65                SoundWave T-Shirt

[66 rows x 1 columns]


In [10]:
# Displaying all unique product categories
unique_categories = dataset["Category"].unique()

# Converting to a DataFrame for a better readability
unique_categories_df = pd.DataFrame(unique_categories, columns=["Category"])

# Displaying the total number of unique categories
print(f"Total Unique Categories: {len(unique_categories)}")

# Displaying the category names
print("\nList of Unique Product Categories:")
print(unique_categories_df)


Total Unique Categories: 4

List of Unique Product Categories:
      Category
0  Electronics
1     Clothing
2        Books
3   Home Decor


In [11]:
# Displaying all unique regions
unique_regions = dataset["Region"].unique()

# Converting to a DataFrame for a better readability
unique_regions_df = pd.DataFrame(unique_regions, columns=["Region"])

# Displaying the total number of unique regions
print(f"Total Unique Regions: {len(unique_regions)}")

# Displaying the region names
print("\nList of Unique Regions:")
print(unique_regions_df)

Total Unique Regions: 4

List of Unique Regions:
          Region
0         Europe
1           Asia
2  South America
3  North America


### Imputation handling

In [12]:
# Checking for missing values in the dataset
missing_values = dataset.isnull().sum()

# Displaying columns with missing values
print("Missing values in the dataset:")
print(missing_values[missing_values > 0])

# Total number of missing values across all columns
total_missing = dataset.isnull().sum().sum()
print(f"\nTotal number of missing values in the dataset: {total_missing}")


Missing values in the dataset:
Series([], dtype: int64)

Total number of missing values in the dataset: 0


- There is no missing values in this dataset.

#### Total Spending by Customer

In [13]:
customer_total_spending = dataset.groupby("CustomerID")["TotalValue"].sum()

print("Total Spending by Customer:")
print(customer_total_spending.head())

Total Spending by Customer:
CustomerID
C0001    3354.52
C0002    1862.74
C0003    2725.38
C0004    5354.88
C0005    2034.24
Name: TotalValue, dtype: float64


- Total spending by the each customer 

#### Transaction Count (Frequency)

In [14]:
customer_transaction_count = dataset.groupby("CustomerID")["TransactionID"].count()

print("\nTransaction Frequency by Customer:")
print(customer_transaction_count.head())


Transaction Frequency by Customer:
CustomerID
C0001    5
C0002    4
C0003    4
C0004    8
C0005    3
Name: TransactionID, dtype: int64


- It gives the number of transactions made by the each customer

#### Average Price of Purchased Products

In [15]:
customer_average_price = dataset.groupby("CustomerID")["Price"].mean()

print("\nAverage Price by Customer:")
print(customer_average_price.head())


Average Price by Customer:
CustomerID
C0001    278.334000
C0002    208.920000
C0003    195.707500
C0004    240.636250
C0005    291.603333
Name: Price, dtype: float64


- It displays the average price of products purchased by each customer.

#### Favourite (Most Purchased) Product Category

In [16]:
customer_favourite_category = dataset.groupby("CustomerID")["Category"].agg(lambda x: x.mode()[0])


print("\nFavourite Category by Customer:")
print(customer_favourite_category.head())


Favourite Category by Customer:
CustomerID
C0001    Electronics
C0002       Clothing
C0003     Home Decor
C0004          Books
C0005    Electronics
Name: Category, dtype: object


- It Shows the most frequently purchased product category.


#### Region

In [17]:
customer_region = dataset.groupby("CustomerID")["Region"].first()

print("\nRegion by Customer:")
print(customer_region.head())


Region by Customer:
CustomerID
C0001    South America
C0002             Asia
C0003    South America
C0004    South America
C0005             Asia
Name: Region, dtype: object


- It displays the region of each customer

In [18]:
# Concatenating customer-level features
customer_features = pd.concat(
    [customer_total_spending, customer_transaction_count, customer_average_price, customer_favourite_category, customer_region], 
    axis=1
)

# Renaming columns to match the original variable names
customer_features.columns = [
    "customer_total_spending", 
    "customer_transaction_count", 
    "customer_average_price", 
    "customer_favourite_category", 
    "customer_region"
]

# Resetting index for better readability
customer_features = customer_features.reset_index()

# Displaying the resulting DataFrame
print("\nCustomer Features:")
print(customer_features.head())



Customer Features:
  CustomerID  customer_total_spending  customer_transaction_count  \
0      C0001                  3354.52                           5   
1      C0002                  1862.74                           4   
2      C0003                  2725.38                           4   
3      C0004                  5354.88                           8   
4      C0005                  2034.24                           3   

   customer_average_price customer_favourite_category customer_region  
0              278.334000                 Electronics   South America  
1              208.920000                    Clothing            Asia  
2              195.707500                  Home Decor   South America  
3              240.636250                       Books   South America  
4              291.603333                 Electronics            Asia  


In [19]:
customer_features.head()

,CustomerID,customer_total_spending,customer_transaction_count,customer_average_price,customer_favourite_category,customer_region
0,C0001,3354.52,5,278.334000,Electronics,South America
1,C0002,1862.74,4,208.920000,Clothing,Asia
2,C0003,2725.38,4,195.707500,Home Decor,South America
3,C0004,5354.88,8,240.636250,Books,South America
4,C0005,2034.24,3,291.603333,Electronics,Asia


This following dataset columns are the most helpful for building a Lookalike model since, they successfully capture the consumer behavior, preferences, and patterns:

#### Useful Columns for Lookalike Model
- **CustomerID**: For the purpose of mapping recommendations, this serves as each customer's primary identifier.  
- **customer_total_spending**: Represents the entire amount of money that the customer has spent. It's a crucial indicator for understanding customer purchasing patterns and business worth.  
- **customer_transaction_count**: The number of transactions indicates how frequently customers make purchases, which is crucial information for figuring out how engaged customers are.  
- **customer_average_price**: A customer's purchasing preferences are indicated by the average price of the goods they buy (e.g., budget-friendly vs. premium).  
- **customer_favourite_category**: The most purchased product category provides information about consumer interests and preferences, which is essential for identifying similar customers.  
- **customer_region**: Due to geographical trends, availability, or tastes, customers from the same region may have similar purchases.  


#### Why These Columns Are Choosen ?  
These columns represents the key aspects of customer behavior:  

- **Spending behavior**: (customer_total_spending, customer_average_price)
- **Purchase frequency**: (customer_transaction_count)
- **Product preferences**: (customer_favourite_category)
- **Geographic influence**: (customer_region)

### Encoding the Categorical Variables

In [20]:
customer_features = pd.get_dummies(customer_features, columns=["customer_region", "customer_favourite_category"], drop_first=True)

In [21]:
customer_features.head()

,CustomerID,customer_total_spending,customer_transaction_count,customer_average_price,customer_region_Europe,customer_region_North America,customer_region_South America,customer_favourite_category_Clothing,customer_favourite_category_Electronics,customer_favourite_category_Home Decor
0,C0001,3354.52,5,278.334000,False,False,True,False,True,False
1,C0002,1862.74,4,208.920000,False,False,False,True,False,False
2,C0003,2725.38,4,195.707500,False,False,True,False,False,True
3,C0004,5354.88,8,240.636250,False,False,True,False,False,False
4,C0005,2034.24,3,291.603333,False,False,False,False,True,False


- Machine learning algorithms and similarity metrics (like cosine similarity) require numerical data.
- One-hot encoding is a technique used to convert categorical data into a binary (0 or 1) numerical format.

In [22]:
print(customer_features.dtypes)


CustomerID                                  object
customer_total_spending                    float64
customer_transaction_count                   int64
customer_average_price                     float64
customer_region_Europe                        bool
customer_region_North America                 bool
customer_region_South America                 bool
customer_favourite_category_Clothing          bool
customer_favourite_category_Electronics       bool
customer_favourite_category_Home Decor        bool
dtype: object


In [23]:
# Identifying the boolean columns
bool_columns = customer_features.select_dtypes(include=['bool']).columns

# Converting boolean columns to the integers
customer_features[bool_columns] = customer_features[bool_columns].astype(int)

# Verifying the updated data types
print(customer_features.dtypes)


CustomerID                                  object
customer_total_spending                    float64
customer_transaction_count                   int64
customer_average_price                     float64
customer_region_Europe                       int64
customer_region_North America                int64
customer_region_South America                int64
customer_favourite_category_Clothing         int64
customer_favourite_category_Electronics      int64
customer_favourite_category_Home Decor       int64
dtype: object


In [24]:
customer_features.head()

,CustomerID,customer_total_spending,customer_transaction_count,customer_average_price,customer_region_Europe,customer_region_North America,customer_region_South America,customer_favourite_category_Clothing,customer_favourite_category_Electronics,customer_favourite_category_Home Decor
0,C0001,3354.52,5,278.334000,0,0,1,0,1,0
1,C0002,1862.74,4,208.920000,0,0,0,1,0,0
2,C0003,2725.38,4,195.707500,0,0,1,0,0,1
3,C0004,5354.88,8,240.636250,0,0,1,0,0,0
4,C0005,2034.24,3,291.603333,0,0,0,0,1,0


- Transforming categorical variables (customer_region,customer_favourite_category) into numeric format for similarity calculations.

### Standardization

In [25]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_features = scaler.fit_transform(customer_features.drop(columns=["CustomerID"]))

In [26]:
scaled_features

array([[-0.06170143, -0.01145819,  0.09467022, ..., -0.54056248,
         1.84992492, -0.51721942],
       [-0.87774353, -0.46749414, -0.90401592, ...,  1.84992492,
        -0.54056248, -0.51721942],
       [-0.40585722, -0.46749414, -1.09410928, ..., -0.54056248,
        -0.54056248,  1.93341543],
       ...,
       [-1.38697529, -1.37956603, -0.46110018, ...,  1.84992492,
        -0.54056248, -0.51721942],
       [-0.81399315, -0.46749414, -0.30420572, ..., -0.54056248,
         1.84992492, -0.51721942],
       [ 0.70636652, -0.01145819,  0.35611784, ...,  1.84992492,
        -0.54056248, -0.51721942]])

Why we had Scaled Features?  
It standardizes the range of numerical data, and ensures:

- All the features be equally contributed to the model.  
- It Avoids the dominance of features with larger ranges over smaller ones.  

**Eg**:  
If **customer_total_spending** ranges in thousands, and **customer_transaction_count** is typically single digits, the larger range might overpower smaller ones in similarity calculations or distance-based models (like cosine similarity or KNN).  


###  Computing Similarity Matrix

In [27]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(scaled_features)

In [28]:
similarity_matrix

array([[ 1.        , -0.30649831,  0.15544862, ..., -0.37235445,
         0.29656692, -0.33432379],
       [-0.30649831,  1.        , -0.06499741, ...,  0.66787063,
        -0.10422184,  0.65882546],
       [ 0.15544862, -0.06499741,  1.        , ..., -0.16580598,
        -0.28750152, -0.38596243],
       ...,
       [-0.37235445,  0.66787063, -0.16580598, ...,  1.        ,
         0.38550092,  0.31606985],
       [ 0.29656692, -0.10422184, -0.28750152, ...,  0.38550092,
         1.        , -0.38385404],
       [-0.33432379,  0.65882546, -0.38596243, ...,  0.31606985,
        -0.38385404,  1.        ]])

In [29]:
# Converting to the DataFrame for readability
import pandas as pd
similarity_df = pd.DataFrame(
    similarity_matrix, 
    index=customer_features["CustomerID"], 
    columns=customer_features["CustomerID"]
)

# Display the first few rows
print(similarity_df.head())


CustomerID     C0001     C0002     C0003     C0004     C0005     C0006  \
CustomerID                                                               
C0001       1.000000 -0.306498  0.155449  0.350298  0.545282  0.444322   
C0002      -0.306498  1.000000 -0.064997 -0.280866  0.027819 -0.296340   
C0003       0.155449 -0.064997  1.000000  0.257611 -0.213255  0.253707   
C0004       0.350298 -0.280866  0.257611  1.000000 -0.441911  0.553383   
C0005       0.545282  0.027819 -0.213255 -0.441911  1.000000 -0.078478   

CustomerID     C0007     C0008     C0009     C0010     C0011     C0012  \
CustomerID                                                               
C0001       0.523149 -0.350408 -0.381519 -0.389408  0.490767  0.154308   
C0002      -0.117335 -0.292210  0.587223  0.703619 -0.229419 -0.422141   
C0003      -0.329459  0.183664 -0.290355 -0.091471  0.371781  0.726597   
C0004      -0.426576  0.172958 -0.582174 -0.331792  0.733597  0.605890   
C0005       0.944697 -0.473082  0.006

Why we used **Cosine Similarity**?  
- It calculates similarity regardless of the vectors' magnitude. This is important when comparing customers with different total spending but similar patterns.  

How to Interpret the Results?  
- **1.0**: Perfectly similar (e.g., customers with identical spending patterns).  
- **0.0**: No similarity (orthogonal vectors, completely different patterns).  
- **-1.0**: Completely opposite patterns (not common for normalized features).  


### Generating the Recommendations

In [30]:
similar_customers = {}
for i, customer_id in enumerate(customer_features["CustomerID"][:20]):
    similarity_scores = list(enumerate(similarity_matrix[i]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[1:4]
    similar_customers[customer_id] = [(customer_features["CustomerID"][j], score) for j, score in similarity_scores]

In [31]:
similar_customers

{'C0001': [('C0181', 0.9829643469753961),
  ('C0192', 0.9535041305694142),
  ('C0190', 0.9503953394872794)],
 'C0002': [('C0088', 0.9764068890164816),
  ('C0134', 0.9463946720184241),
  ('C0106', 0.9463736594511042)],
 'C0003': [('C0025', 0.9804399773534254),
  ('C0031', 0.9733680506789716),
  ('C0052', 0.9687586191651847)],
 'C0004': [('C0165', 0.970907661586851),
  ('C0153', 0.9214483777482797),
  ('C0087', 0.9111137479183313)],
 'C0005': [('C0186', 0.9884948596671941),
  ('C0140', 0.9880089510695753),
  ('C0146', 0.9450658915313815)],
 'C0006': [('C0171', 0.9531784721207105),
  ('C0011', 0.9390052505353642),
  ('C0168', 0.9379172027604062)],
 'C0007': [('C0146', 0.9849330171639457),
  ('C0115', 0.9789839666276232),
  ('C0186', 0.9723725231037348)],
 'C0008': [('C0065', 0.892147853584243),
  ('C0059', 0.7860395194334052),
  ('C0160', 0.7698077613027499)],
 'C0009': [('C0061', 0.9791075049919215),
  ('C0198', 0.9594425561301161),
  ('C0062', 0.9239737736032363)],
 'C0010': [('C0111', 

- We had found the top 3 most similar customers based on similarity scores.
- The output results in a dictionary mapping of each customer to their top 3 lookalikes with similarity scores.

### Saving the results to CSV

In [32]:
lookalike_results = []
for cust_id, similar_list in similar_customers.items():
    for similar_cust, score in similar_list:
        lookalike_results.append({"CustomerID": cust_id, "SimilarCustomerID": similar_cust, "Score": score})

lookalike_df = pd.DataFrame(lookalike_results)
lookalike_df.to_csv("Lookalike.csv", index=False)

print("Lookalike.csv generated successfully!")

Lookalike.csv generated successfully!
